In [2]:
import pandas as pd
import numpy as np
import os


# Scrap book

This ipynb, is for all the wierd fiddly bits you need to have. It is not in marimo as it probably contains many unrelated bits that could take a while to run.

This is annoying but, it is life

## Making Ground truth

Because there are so many bounding boxes in the NCSE test set I am using the response from pixtral to create the initial GT text files. I can then replace the text in them with the actual GT from the transcriptions. This should make it faster as I can read what is in there.


In [5]:


pixtral_ncse_test_response_path = 'data/download_jobs/experiments/dataframe/NCSE_deskew_True_max_ratio_1.csv'

pixtral_response_df = pd.read_csv(pixtral_ncse_test_response_path)

bbox_transcripts_path = 'data/transcripts/bbox_pixtral'

os.makedirs(bbox_transcripts_path, exist_ok=True)

for index, row in pixtral_response_df.iterrows():
    with open(os.path.join(bbox_transcripts_path , row['filename']), 'w') as file:  # 'w' for text mode
        file.write(row['content'])


bbox_transcripts_path = 'data/model_performance/NCSE_pixtral'

os.makedirs(bbox_transcripts_path, exist_ok=True)
# Rename the files so that it harmonsies with the other file names.. it is annnouying this has to be to done but is not important enough to fix
pixtral_response_df['filename'] = pixtral_response_df['filename'].str.replace("_box_page_id", "")
for index, row in pixtral_response_df.iterrows():
    with open(os.path.join(bbox_transcripts_path , row['filename']), 'w') as file:  # 'w' for text mode
        file.write(row['content'])

In [17]:


pixtral_BLN_test_response_path = 'data/download_jobs/experiments/dataframe/BLN600_deskew_True_max_ratio_1.csv'

pixtral_response_df = pd.read_csv(pixtral_BLN_test_response_path)

bbox_transcripts_path = 'data/model_performance/BLN_pixtral'

os.makedirs(bbox_transcripts_path, exist_ok=True)
# Rename the files so that it harmonsies with the other file names.. it is annnouying this has to be to done but is not important enough to fix
pixtral_response_df['filename'] = pixtral_response_df['issue_id'].astype(str) + '.txt'
for index, row in pixtral_response_df.iterrows():
    with open(os.path.join(bbox_transcripts_path , row['filename']), 'w') as file:  # 'w' for text mode
        file.write(row['content'])

In [15]:
pixtral_response_df

,issue_id,page_number,block,column,reading_order,content,segment_count,prompt_tokens,completion_tokens,total_tokens,box_page_id,page_id,filename
0,3200797029,1,0,1,0,OR SEVEN DAYS.\n\nROBBERY AT A BARONET'S.\n\nE...,2,5390,610,6000,B0C1R0,3200797029_page_1,3200797029_page_1_box_page_id_B0C1R0.txt
1,3200797032,1,0,1,0,CHARGE OF SETTING FIRE TO A HOUSE.\n\nAt the T...,2,5186,628,5814,B0C1R0,3200797032_page_1,3200797032_page_1_box_page_id_B0C1R0.txt
2,3200797034,1,0,1,0,SERVANTS AND THEIR SWEETHEARTS.\n\nOn Saturday...,3,8397,899,9296,B0C1R0,3200797034_page_1,3200797034_page_1_box_page_id_B0C1R0.txt
3,3200797037,1,0,1,0,CAPTURE OF PICKPOCKETS.\n\nAt Stratford petty ...,3,7185,808,7993,B0C1R0,3200797037_page_1,3200797037_page_1_box_page_id_B0C1R0.txt
4,3200801612,1,0,1,0,DOUBLE MURDER BY A MOTHER AT RUSHOLME.\n\n[SUB...,5,14525,1427,15952,B0C1R0,3200801612_page_1,3200801612_page_1_box_page_id_B0C1R0.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,3207643520,1,0,1,0,```\nLAW NOTICES.—This Day.\n\nCOURT OF CHANCE...,2,6246,920,7166,B0C1R0,3207643520_page_1,3207643520_page_1_box_page_id_B0C1R0.txt
596,3207644658,1,0,1,0,SOUTH DEVON ELECTION.\n\nTo the EDITOR of the ...,3,8397,834,9231,B0C1R0,3207644658_page_1,3207644658_page_1_box_page_id_B0C1R0.txt
597,3207644853,1,0,1,0,```\nMIRROR OF FASHION.\n\n[FROM THE COURT CIR...,3,8397,840,9237,B0C1R0,3207644853_page_1,3207644853_page_1_box_page_id_B0C1R0.txt
598,3207646187,1,0,1,0,LONDON MARKETS.—MONDAY.\n\nCORN EXCHANGE.\n\nT...,1,2695,162,2857,B0C1R0,3207646187_page_1,3207646187_page_1_box_page_id_B0C1R0.txt


## Chronicalling Germany 

This provides a quick look at the Chroncialling Germany dataset. It could be added as a curve ball dataextraction if needed. The use of fraktur font and and German will result in higher errors which are a good way to introduce fine-tuning. However, it is a bunch more work so I am not super stoked by the idea. The data should also probably be pre-proscess to make it smaller as it is currently 4.5 gb for 400 pages.







In [27]:
import xml.etree.ElementTree as ET

def parse_coords(points_str):
    """Convert points string to list of coordinates"""
    points = points_str.strip().split()
    coords = []
    for point in points:
        x, y = point.split(',')
        coords.append((int(x), int(y)))
    return coords

def get_bounding_box(coords):
    """Calculate bounding box (min_x, min_y, max_x, max_y) from coordinates"""
    x_coords = [x for x, y in coords]
    y_coords = [y for x, y in coords]
    return {
        'x1': min(x_coords),
        'y1': min(y_coords),
        'x2': max(x_coords),
        'y2': max(y_coords)
    }

def extract_text_and_boxes(xml_string):
    # Parse XML
    root = ET.fromstring(xml_string)
    
    # Define namespace
    namespace = {'ns': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}
    
    # Dictionary to store all document information
    document_info = {
        'regions': {},
        'metadata': {
            'n_regions': 0,
            'region_types': set()
        }
    }
    
    # Find all TextRegion elements
    for region in root.findall('.//ns:TextRegion', namespace):
        region_id = region.get('id', 'unknown')
        
        # Get region type and custom attributes
        region_type = region.get('type', 'unknown')
        custom_attr = region.get('custom', '')
        
        # Extract reading order index
        reading_order = None
        if 'readingOrder {index:' in custom_attr:
            try:
                reading_order = int(custom_attr.split('readingOrder {index:')[1].split('}')[0])
            except:
                reading_order = 999
        
        # Get coordinates
        coords_elem = region.find('.//ns:Coords', namespace)
        if coords_elem is not None:
            points = coords_elem.get('points')
            coords = parse_coords(points)
            bbox = get_bounding_box(coords)
        else:
            bbox = None
        
        # Get text from all lines in region
        region_text = []
        for text_line in region.findall('.//ns:TextLine', namespace):
            unicode_elem = text_line.find('.//ns:Unicode', namespace)
            if unicode_elem is not None and unicode_elem.text:
                region_text.append(unicode_elem.text.strip())
        
        # Store region information
        document_info['regions'][region_id] = {
            'reading_order': reading_order,
            'type': region_type,
            'bbox': bbox,
            'text': '\n'.join(region_text),
            'raw_coords': coords if coords_elem is not None else None
        }
        
        # Update metadata
        document_info['metadata']['region_types'].add(region_type)
    
    # Update final metadata
    document_info['metadata']['n_regions'] = len(document_info['regions'])
    document_info['metadata']['region_types'] = list(document_info['metadata']['region_types'])
    
    return document_info


In [36]:

xml_file = "/home/jonno/ereading_the_unreadable/data/Berliner_Boersen_Zeitung_1857-04-06_0001.xml"

# Usage example:
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_content = file.read()
    
doc_info = extract_text_and_boxes(xml_content)

# Example: Convert to DataFrame
import pandas as pd

def create_bbox_dataframe(doc_info):
    rows = []
    for region_id, region_data in doc_info['regions'].items():
        if region_data['bbox']:
            row = {
                'region_id': region_id,
                'type': region_data['type'],
                'reading_order': region_data['reading_order'],
                **region_data['bbox']  # Unpacks x1, y1, x2, y2
            }
            rows.append(row)
    return pd.DataFrame(rows)

# Create DataFrame of bounding boxes
bbox_df = create_bbox_dataframe(doc_info)

# Example usage:
print("\nDocument Metadata:")
print(doc_info['metadata'])

print("\nSample region data:")
sample_region_id = list(doc_info['regions'].keys())[0]
print(doc_info['regions'][sample_region_id])

print("\nBounding Box DataFrame:")
print(bbox_df.head())


Document Metadata:
{'n_regions': 18, 'region_types': ['heading', 'header', 'unknown', 'caption']}

Sample region data:
{'reading_order': 999, 'type': 'caption', 'bbox': {'x1': 165, 'y1': 209, 'x2': 797, 'y2': 901}, 'text': 'HrinInama\nd\nDieses Blatt erscheint ohne Anmnahme\ntäglich zweimal.\nAbonnemeuts-Prei-\nvierteljährl. für Berlin 2 Thlr 15Sgr.,\nfur gan Preussen 3 Thlr., für ganz\nDeutschland 3 Thlr. 15\nIusertious-Gebühr\ntar cte dreigespaltene Zeile 2gr.', 'raw_coords': [(172, 764), (165, 400), (213, 209), (739, 209), (787, 378), (776, 461), (772, 547), (797, 582), (789, 669), (763, 728), (787, 895), (178, 901)]}

Bounding Box DataFrame:
                  region_id     type  reading_order   x1    y1    x2    y2
0   region_1722350650121_61  caption            999  165   209   797   901
1  region_1722350869520_107  heading            999  177  1071  1138  1161
2  region_1722350991056_123  unknown            999  165  1147  1139  1566
3  region_1722350874930_110  heading         

In [38]:
doc_info['regions']

{'region_1722350650121_61': {'reading_order': 999,
  'type': 'caption',
  'bbox': {'x1': 165, 'y1': 209, 'x2': 797, 'y2': 901},
  'text': 'HrinInama\nd\nDieses Blatt erscheint ohne Anmnahme\ntäglich zweimal.\nAbonnemeuts-Prei-\nvierteljährl. für Berlin 2 Thlr 15Sgr.,\nfur gan Preussen 3 Thlr., für ganz\nDeutschland 3 Thlr. 15\nIusertious-Gebühr\ntar cte dreigespaltene Zeile 2gr.',
  'raw_coords': [(172, 764),
   (165, 400),
   (213, 209),
   (739, 209),
   (787, 378),
   (776, 461),
   (772, 547),
   (797, 582),
   (789, 669),
   (763, 728),
   (787, 895),
   (178, 901)]},
 'region_1722350869520_107': {'reading_order': 999,
  'type': 'heading',
  'bbox': {'x1': 177, 'y1': 1071, 'x2': 1138, 'y2': 1161},
  'text': 'GeneßehendHerGär.itg',
  'raw_coords': [(177, 1071),
   (1138, 1089),
   (1137, 1150),
   (1118, 1161),
   (1066, 1145),
   (461, 1135),
   (441, 1149),
   (425, 1135),
   (334, 1137),
   (307, 1150),
   (284, 1147),
   (268, 1136),
   (179, 1142)]},
 'region_1722350991056_123